In [2]:
!pip install langchain qdrant-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver doe

# **Tải về văn bản Sổ tay sinh viên 2023**

In [3]:
!gdown https://drive.google.com/uc?id=1AYOtbqRekHC5kEs7uckcC3MEcWk4VwF9 -O sotay.txt

Downloading...
From: https://drive.google.com/uc?id=1AYOtbqRekHC5kEs7uckcC3MEcWk4VwF9
To: /content/sotay.txt
100% 107k/107k [00:00<00:00, 150MB/s]


# **Xử lý dữ liệu**

In [4]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader,TextLoader
# Load file dữ liệu sổ tay sinh viên đã được chuẩn hóa
loader_sotay = TextLoader("sotay.txt")

In [25]:
# Cắt pdf thành các đoạn text nhỏ, size là 1000 mỗi đoạn
documents_sotay = loader_sotay.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                                chunk_overlap=400)

texts_sotay = text_splitter.split_documents(documents_sotay)


In [26]:
len(texts_sotay)

184

In [18]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = "<QDRANT_API_KEY>"
QDRANT_URL = "<QDRANT_URL>"
HUGGINGFACE_API_KEY= "<HUGGINGFACE_API_KEY>"

In [19]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# Tạo lớp embeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    api_key = HUGGINGFACE_API_KEY,
    model_kwargs = {'device': 'auto'}
)

# Encode dữ liệu text qua lớp embedding,
# rồi lưu lại trên vector database server
qdrant = Qdrant.from_documents(
    texts_sotay,
    embeddings,
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="nttu_sotay_vector_db_v1",
    api_key=QDRANT_API_KEY,
)

print("Tạo vector database thành công!")

Tạo vector database thành công!


In [23]:
# Test tìm văn bản liên quan đến query
query = "Xếp loại học bổng ở trường"
found_docs = qdrant.similarity_search(query,k=15)

In [24]:
found_docs

[Document(page_content='trong việc xem xét tặng học bổng xã hội mỗi học kỳ. Miễn phí gửi xe tại các cơ sở của Trường. Được xem xét ưu tiên giảm học phí khi đăng ký các lớp kỹ năng do Nhà Trường tổ chức. Được nhận phụ cấp theo quy định Nhà trường cụ thể: + Đối với Bí thư Chi đoàn kiêm nhiệm Lớp trưởng hoàn thành nhiệm vụ được nhận phụ cấp theo mức 200.000đ/tháng/cá nhân x 10 tháng. + Đối với Chi Hội trưởng kiêm nhiệm Lớp phó hoàn thành nhiệm vụ được nhận phụ cấp theo mức 100.000đ/tháng/cá', metadata={'source': 'sotay.txt'}),
 Document(page_content='mức độ hoàn thành nhiệm vụ trở lên sẽ được hưởng quyền lợi và chế độ như sau: Được cộng điểm vào điểm đánh giá kết quả rèn luyện cuối mỗi học kỳ, điểm phân tích chất lượng đoàn viên, hội viên vào cuối mỗi năm học. Được ưu tiên trong việc xem xét tặng học bổng xã hội mỗi học kỳ. Miễn phí gửi xe tại các cơ sở của Trường. Được xem xét ưu tiên giảm học phí khi đăng ký các lớp kỹ năng do Nhà Trường tổ chức. Được nhận phụ cấp theo quy định Nhà', me